# Mobile Customer Churn

In this Portfolio task you will work with some (fake but realistic) data on Mobile Customer Churn.  Churn is where
a customer leaves the mobile provider.   The goal is to build a simple predictive model to predict churn from available features. 

The data was generated (by Hume Winzar at Macquarie) based on a real dataset provided by Optus.  The data is simulated but the column headings are the same. (Note that I'm not sure if all of the real relationships in this data are preserved so you need to be cautious in interpreting the results of your analysis here).  

The data is provided in file `MobileCustomerChurn.csv` and column headings are defined in a file `MobileChurnDataDictionary.csv` (store these in the `files` folder in your project).

Your high level goal in this notebook is to try to build and evaluate a __predictive model for churn__ - predict the value of the CHURN_IND field in the data from some of the other fields.  Note that the three `RECON` fields should not be used as they indicate whether the customer reconnected after having churned. 

__Note:__ you are not being evaluated on the _accuracy_ of the model but on the _process_ that you use to generate it.  You can use a simple model such as Logistic Regression for this task or try one of the more advanced methods covered in recent weeks.  Explore the data, build a model using a selection of features and then do some work on finding out which features provide the most accurate results.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE

In [2]:
churn = pd.read_csv("files/MobileCustomerChurn.csv", na_values=["NA", "#VALUE!"], index_col='INDEX')
churn.head()

,CUST_ID,ACCOUNT_TENURE,ACCT_CNT_SERVICES,AGE,CFU,SERVICE_TENURE,PLAN_ACCESS_FEE,BYO_PLAN_STATUS,PLAN_TENURE,MONTHS_OF_CONTRACT_REMAINING,...,CONTRACT_STATUS,PREV_CONTRACT_DURATION,HANDSET_USED_BRAND,CHURN_IND,MONTHLY_SPEND,COUNTRY_METRO_REGION,STATE,RECON_SMS_NEXT_MTH,RECON_TELE_NEXT_MTH,RECON_EMAIL_NEXT_MTH
INDEX,,,,,,,,,,,,,,,,,,,,,
1,1,46,1,30.0,CONSUMER,46,54.54,NON BYO,15,0,...,OFF-CONTRACT,24,SAMSUNG,1,61.40,COUNTRY,WA,NaN,NaN,NaN
2,2,60,3,55.0,CONSUMER,59,54.54,NON BYO,5,0,...,OFF-CONTRACT,24,APPLE,1,54.54,METRO,NSW,NaN,NaN,NaN
3,5,65,1,29.0,CONSUMER,65,40.90,BYO,15,0,...,OFF-CONTRACT,12,APPLE,1,2.50,COUNTRY,WA,NaN,NaN,NaN
4,6,31,1,51.0,CONSUMER,31,31.81,NON BYO,31,0,...,OFF-CONTRACT,24,APPLE,1,6.48,COUNTRY,VIC,NaN,NaN,NaN
5,8,95,1,31.0,CONSUMER,95,54.54,NON BYO,0,0,...,OFF-CONTRACT,24,APPLE,1,100.22,METRO,NSW,NaN,NaN,NaN


#  Exploring the Dataset

In [3]:
churn.shape

(46206, 21)

In [4]:
churn.describe()

,CUST_ID,ACCOUNT_TENURE,ACCT_CNT_SERVICES,AGE,SERVICE_TENURE,PLAN_ACCESS_FEE,PLAN_TENURE,MONTHS_OF_CONTRACT_REMAINING,LAST_FX_CONTRACT_DURATION,PREV_CONTRACT_DURATION,CHURN_IND,MONTHLY_SPEND,RECON_SMS_NEXT_MTH,RECON_TELE_NEXT_MTH,RECON_EMAIL_NEXT_MTH
count,46206.000000,46206.000000,46206.000000,46130.000000,46206.000000,46206.000000,46206.000000,46206.000000,46206.000000,46206.000000,46206.000000,46206.000000,28416.000000,28416.000000,28416.000000
mean,42335.196879,45.874627,1.555707,41.411836,50.318617,51.364858,10.846708,8.237112,20.349825,15.249968,0.385015,75.188029,0.014640,0.191125,0.007038
std,22100.432751,33.077927,0.835529,15.263726,51.925537,20.855419,9.769887,8.341476,8.034631,10.982958,0.486604,73.519041,0.120108,0.393194,0.083600
min,1.000000,0.000000,1.000000,-4.000000,0.000000,8.180000,0.000000,0.000000,0.000000,0.000000,0.000000,1.020000,0.000000,0.000000,0.000000
25%,24951.750000,14.000000,1.000000,28.000000,11.000000,36.360000,3.000000,0.000000,24.000000,0.000000,0.000000,36.360000,0.000000,0.000000,0.000000
50%,43261.000000,44.000000,1.000000,40.000000,35.000000,54.540000,8.000000,7.000000,24.000000,24.000000,0.000000,54.540000,0.000000,0.000000,0.000000
75%,61132.750000,77.000000,2.000000,52.000000,69.000000,72.720000,16.000000,16.000000,24.000000,24.000000,1.000000,84.530000,0.000000,0.000000,0.000000
max,79500.000000,120.000000,4.000000,116.000000,259.000000,234.540000,147.000000,24.000000,36.000000,36.000000,1.000000,1965.890000,1.000000,1.000000,1.000000


In [5]:
churn.value_counts('CHURN_IND')

CHURN_IND
0    28416
1    17790
dtype: int64

In [6]:
#Extremely Laggy
#sns.pairplot(data = churn.sample(1000), hue = 'CHURN_IND')

## Regression Data Preparation

Removal of irrelevant columns based on data/problem context

In [7]:
churn = churn.drop(['RECON_SMS_NEXT_MTH', 'RECON_TELE_NEXT_MTH', 'RECON_EMAIL_NEXT_MTH'], axis = 1)

NaN value check with apporopriate deletion of rows

In [8]:
churn.isna().sum()
churn = churn.dropna(axis=0)

In [9]:
churn.head()

,CUST_ID,ACCOUNT_TENURE,ACCT_CNT_SERVICES,AGE,CFU,SERVICE_TENURE,PLAN_ACCESS_FEE,BYO_PLAN_STATUS,PLAN_TENURE,MONTHS_OF_CONTRACT_REMAINING,LAST_FX_CONTRACT_DURATION,CONTRACT_STATUS,PREV_CONTRACT_DURATION,HANDSET_USED_BRAND,CHURN_IND,MONTHLY_SPEND,COUNTRY_METRO_REGION,STATE
INDEX,,,,,,,,,,,,,,,,,,
1,1,46,1,30.0,CONSUMER,46,54.54,NON BYO,15,0,24,OFF-CONTRACT,24,SAMSUNG,1,61.40,COUNTRY,WA
2,2,60,3,55.0,CONSUMER,59,54.54,NON BYO,5,0,24,OFF-CONTRACT,24,APPLE,1,54.54,METRO,NSW
3,5,65,1,29.0,CONSUMER,65,40.90,BYO,15,0,12,OFF-CONTRACT,12,APPLE,1,2.50,COUNTRY,WA
4,6,31,1,51.0,CONSUMER,31,31.81,NON BYO,31,0,24,OFF-CONTRACT,24,APPLE,1,6.48,COUNTRY,VIC
5,8,95,1,31.0,CONSUMER,95,54.54,NON BYO,0,0,24,OFF-CONTRACT,24,APPLE,1,100.22,METRO,NSW


Identification of potential 2 value categorical columns

In [10]:
churn.value_counts('COUNTRY_METRO_REGION')


COUNTRY_METRO_REGION
METRO      31765
COUNTRY    14364
dtype: int64

In [11]:
churn.value_counts('CFU')

CFU
CONSUMER          39078
SMALL BUSINESS     7051
dtype: int64

In [12]:
churn.value_counts('BYO_PLAN_STATUS')

BYO_PLAN_STATUS
NON BYO    35412
BYO        10717
dtype: int64

In [13]:
churn.value_counts('CONTRACT_STATUS')

CONTRACT_STATUS
ON-CONTRACT     28233
OFF-CONTRACT    12443
NO-CONTRACT      5453
dtype: int64

Conversion of identified 2 value categorical columns into numerical booleans

In [14]:
churn['Is Metro'] = (churn['COUNTRY_METRO_REGION'] =='METRO').astype(int)

In [15]:
churn['is BYO'] = (churn['BYO_PLAN_STATUS'] == 'BYO').astype(int)

In [16]:
churn['is CONSUMER'] = (churn['CFU'] == 'CONSUMER').astype(int)

Conversion of 3+ value categorical columns into numerical booleans

In [17]:
multi_df = pd.get_dummies(churn['CONTRACT_STATUS'])
churn = pd.concat([churn, multi_df], axis = 1)

In [18]:
multi_df = pd.get_dummies(churn['HANDSET_USED_BRAND'])
churn = pd.concat([churn, multi_df], axis = 1)
churn.head()

,CUST_ID,ACCOUNT_TENURE,ACCT_CNT_SERVICES,AGE,CFU,SERVICE_TENURE,PLAN_ACCESS_FEE,BYO_PLAN_STATUS,PLAN_TENURE,MONTHS_OF_CONTRACT_REMAINING,...,is CONSUMER,NO-CONTRACT,OFF-CONTRACT,ON-CONTRACT,APPLE,GOOGLE,HUAWEI,OTHER,SAMSUNG,UNKNOWN
INDEX,,,,,,,,,,,,,,,,,,,,,
1,1,46,1,30.0,CONSUMER,46,54.54,NON BYO,15,0,...,1,0,1,0,0,0,0,0,1,0
2,2,60,3,55.0,CONSUMER,59,54.54,NON BYO,5,0,...,1,0,1,0,1,0,0,0,0,0
3,5,65,1,29.0,CONSUMER,65,40.90,BYO,15,0,...,1,0,1,0,1,0,0,0,0,0
4,6,31,1,51.0,CONSUMER,31,31.81,NON BYO,31,0,...,1,0,1,0,1,0,0,0,0,0
5,8,95,1,31.0,CONSUMER,95,54.54,NON BYO,0,0,...,1,0,1,0,1,0,0,0,0,0


In [19]:
multi_df = pd.get_dummies(churn['STATE'])
churn = pd.concat([churn, multi_df], axis = 1)

In [20]:
churn.head()

,CUST_ID,ACCOUNT_TENURE,ACCT_CNT_SERVICES,AGE,CFU,SERVICE_TENURE,PLAN_ACCESS_FEE,BYO_PLAN_STATUS,PLAN_TENURE,MONTHS_OF_CONTRACT_REMAINING,...,SAMSUNG,UNKNOWN,ACT,NSW,NT,QLD,SA,TAS,VIC,WA
INDEX,,,,,,,,,,,,,,,,,,,,,
1,1,46,1,30.0,CONSUMER,46,54.54,NON BYO,15,0,...,1,0,0,0,0,0,0,0,0,1
2,2,60,3,55.0,CONSUMER,59,54.54,NON BYO,5,0,...,0,0,0,1,0,0,0,0,0,0
3,5,65,1,29.0,CONSUMER,65,40.90,BYO,15,0,...,0,0,0,0,0,0,0,0,0,1
4,6,31,1,51.0,CONSUMER,31,31.81,NON BYO,31,0,...,0,0,0,0,0,0,0,0,1,0
5,8,95,1,31.0,CONSUMER,95,54.54,NON BYO,0,0,...,0,0,0,1,0,0,0,0,0,0


Removal of non numerical columns

In [21]:
churn = churn.drop(['COUNTRY_METRO_REGION', 'BYO_PLAN_STATUS', 'CFU',
                    'CONTRACT_STATUS', 'HANDSET_USED_BRAND', 'STATE'], axis = 1)

# Logistic Regression

Data Split into a 20-80 split for test and train sets.

In [22]:
train, test = train_test_split(churn, test_size = 0.2)
print(train.shape)
print(test.shape)

(36903, 32)
(9226, 32)


Logistic Regression Model

In [23]:
X_train = train.drop(['CHURN_IND','CUST_ID',], axis = 1)
y_train = train['CHURN_IND']
X_test = test.drop(['CHURN_IND','CUST_ID'], axis = 1)
y_test = test['CHURN_IND']

In [26]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [27]:
lr.coef_

array([[-6.61470378e-03, -7.85236043e-03, -1.65812192e-02,
        -4.95646917e-03, -5.66037289e-03,  7.45370958e-03,
        -8.89531234e-02,  1.67012670e-02, -1.33376429e-02,
         1.17117499e-02, -3.03238911e-01, -7.11521606e-01,
         1.20804266e-01,  4.40681568e-01,  7.51682404e-01,
        -4.53291901e-01, -1.38131267e-01, -6.71326622e-02,
        -3.04626534e-02,  2.46958454e-01,  4.20761186e-04,
         7.27419439e-01, -6.09554767e-02,  1.93071374e-01,
        -2.38038362e-03,  2.30586692e-01,  6.78153885e-02,
        -3.43908174e-02,  1.43462811e-01,  2.01862484e-01]])

# Evaluation

In [28]:
train_preds = lr.predict(X_train)
test_preds = lr.predict(X_test)
print("Train Accuracy:")
print(accuracy_score(y_train, train_preds))
print("Test Accuracy:")
print(accuracy_score(y_test, test_preds))

Train Accuracy:
0.7259301411809338
Test Accuracy:
0.7220897463689573


Test Accuracy is quite close to that of the Train accuracy indicating that is no real issue of overfitting in the logistic regression model

Confusion Matrix

In [29]:
print("Confusion matrix on test set: ")
print(confusion_matrix(y_test, test_preds))

Confusion matrix on test set: 
[[4786  953]
 [1611 1876]]


# Feature Selection and Optimization

In [30]:
lr_model = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=lr_model, n_features_to_select=5, step=1)
rfe.fit(X_train, y_train)

RFE(estimator=LogisticRegression(max_iter=1000), n_features_to_select=5)

In [31]:
y_test_hat = rfe.predict(X_test)
print("accuracy score on test set: ", accuracy_score(y_test, y_test_hat))

accuracy score on test set:  0.6673531324517668


In [32]:
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected False, Rank: 19.000
Column: 1, Selected False, Rank: 5.000
Column: 2, Selected False, Rank: 17.000
Column: 3, Selected False, Rank: 24.000
Column: 4, Selected False, Rank: 23.000
Column: 5, Selected False, Rank: 22.000
Column: 6, Selected False, Rank: 11.000
Column: 7, Selected False, Rank: 12.000
Column: 8, Selected False, Rank: 14.000
Column: 9, Selected False, Rank: 21.000
Column: 10, Selected True, Rank: 1.000
Column: 11, Selected True, Rank: 1.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected False, Rank: 4.000
Column: 14, Selected False, Rank: 13.000
Column: 15, Selected True, Rank: 1.000
Column: 16, Selected False, Rank: 15.000
Column: 17, Selected False, Rank: 2.000
Column: 18, Selected False, Rank: 8.000
Column: 19, Selected False, Rank: 3.000
Column: 20, Selected False, Rank: 26.000
Column: 21, Selected True, Rank: 1.000
Column: 22, Selected False, Rank: 18.000
Column: 23, Selected False, Rank: 9.000
Column: 24, Selected False, Rank: 25.000
C

## Accuracy with addtion of features

In [35]:
# Extremely laggy
acc_scores = []
for i in range(1,30):
    clf = LogisticRegression(max_iter=1000)
    rfe = RFE(estimator=clf, n_features_to_select=i)
    # training model
    rfe.fit(X_train, y_train)
    # predicting on test set
    y_pred = rfe.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    # print this
    print("Acc on test set using", i, "features: ", acc_score)
    # append to the list
    acc_scores.append(acc_score)

Acc on test set using 1 features:  0.6365705614567526
Acc on test set using 2 features:  0.6365705614567526
Acc on test set using 3 features:  0.6745068285280729
Acc on test set using 4 features:  0.6646433990895296
Acc on test set using 5 features:  0.6673531324517668
Acc on test set using 6 features:  0.6671363537827878
Acc on test set using 7 features:  0.6670279644482983
Acc on test set using 8 features:  0.6675699111207457
Acc on test set using 9 features:  0.6686538044656406
Acc on test set using 10 features:  0.669195751138088
Acc on test set using 11 features:  0.6689789724691091
Acc on test set using 12 features:  0.6689789724691091
Acc on test set using 13 features:  0.6715803164968567
Acc on test set using 14 features:  0.6714719271623673
Acc on test set using 15 features:  0.6695209191415564
Acc on test set using 16 features:  0.6681118577931932
Acc on test set using 17 features:  0.6682202471276827
Acc on test set using 18 features:  0.6720138738348147
Acc on test set usin

Based on these results the best results are achieved by utilising all features as opposed to limiting features.